In [0]:
# IMPORTS
import sys
import os
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'src'))

import yaml
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession

import plotly.express as px

# Custom funcs
from data_pull.loaders import (
    load_user_table,
    load_task_complete_table,
    load_respondent_info_table,
    load_task_table,
    load_all_wonky_studies
)
from data_pull.joiners import (
    join_user_task_respondent,
    join_wonky_balance_with_task,
    merge_wonky_data_with_user_info
)
from data_pull.aggregators import (
    enrich_user_info_with_task_counts,
    union_wonky_study_dataframes,
    aggregate_wonky_respondent_summary,
    create_wonky_respondent_summary,
    calculate_wonky_task_ratio
)

# Configs
with open('../configs/data_paths.yaml', 'r') as f:
    paths_config = yaml.safe_load(f)

with open('../configs/wonky_studies.yaml', 'r') as f:
    wonky_config = yaml.safe_load(f)

print("Imports and configs loaded successfully")


In [0]:
# needed on each re-run
spark.catalog.clearCache()
print("Cache cleared")

In [0]:
# loading the tables
s_u = load_user_table(
    spark,
    paths_config['silver_path'],
    country=paths_config['filters']['country']
)

s_tc_odr = load_task_complete_table(
    spark,
    paths_config['silver_path'],
    min_date=paths_config['filters']['min_date'],
    task_origin=paths_config['filters']['task_origin']
)

s_ri = load_respondent_info_table(
    spark,
    paths_config['silver_path'],
    country=paths_config['filters']['country']
)

print(f"s_u (users {paths_config['filters']['country']}) count: {s_u.count():,}")
print(f"s_tc_odr (tasks completed >= {paths_config['filters']['min_date']}, origin={paths_config['filters']['task_origin']}) count: {s_tc_odr.count():,}")
print(f"s_ri (respondent_info {paths_config['filters']['country']}) count: {s_ri.count():,}")


In [0]:
sorted((s_tc_odr.toPandas()).columns)

In [0]:
print(f"\ns_u unique respondent_pk: {s_u.select('respondent_pk').distinct().count():,}")
print(f"s_ri unique respondent_pk: {s_ri.select('respondent_pk').distinct().count():,}")

In [0]:
print(f"s_u (users GB) count: {s_u.count():,}")
print(f"s_tc_odr (tasks completed >= 2025-09-11, origin=odr) count: {s_tc_odr.count():,}")
print(f"s_ri (respondent_info GB) count: {s_ri.count():,}")

In [0]:
# join and print
user_info = join_user_task_respondent(s_u, s_tc_odr, s_ri)

print(f"\nAfter INNER joins count: {user_info.count():,}")
print(f"Unique respondents in joined data: {user_info.select('respondentPk').distinct().count():,}")

In [0]:
# enrich >> consistent with original notebook
user_info_enriched = enrich_user_info_with_task_counts(user_info)
user_info_enriched.cache()

print(f"Task-level records (rows): {user_info_enriched.count():,}")
print(f"Unique respondents: {user_info_enriched.select('respondentPk').distinct().count():,}")

In [0]:
user_info_enriched.toPandas()

In [0]:
user_info_df = user_info_enriched.toPandas()

print(f"\nHierarchical Structure Validation:")
print(f"  - Task-level rows: {len(user_info_df):,}")
print(f"  - Unique respondents: {user_info_df['respondentPk'].nunique():,}")
print(f"  - Avg tasks per respondent: {len(user_info_df) / user_info_df['respondentPk'].nunique():.2f}")

In [0]:
user_info_df[['respondentPk', 'task_completed']].groupby('respondentPk').sum().sort_values(by='task_completed')

In [0]:
sorted(user_info_df.columns)

In [0]:
user_info_df['fulcrum_household_income'].unique()

In [0]:
user_info_df['gambling_participation_mc']

In [0]:
s = user_info_df['gambling_participation_mc']

# One-hot encode each gambling mode
dummies = (
    s.explode()                    
     .str.strip()                   
     .pipe(pd.get_dummies)          
     .groupby(level=0).sum()     
)

In [0]:
user_info_df.columns

In [0]:
# retrieve wonky studies from balance tables
balance_dfs, failed_uuids = load_all_wonky_studies(
    spark,
    wonky_config['wonky_study_uuids'],
    base_path=paths_config['project_repository_path'],
    cols_to_include=wonky_config['cols_to_include_subset'],
    verbose=True
)

print(f"\nSuccessfully loaded {len(balance_dfs)} out of {len(wonky_config['wonky_study_uuids'])} studies")
if failed_uuids:
    print(f"Failed UUIDs ({len(failed_uuids)}): {failed_uuids}")


In [0]:
# join wonky studies with task table
task = load_task_table(spark, paths_config['silver_path'])

wonky_dfs_joined = []
for balance_df in balance_dfs:
    joined = join_wonky_balance_with_task(balance_df, task)
    wonky_dfs_joined.append(joined)

wonky_spark = union_wonky_study_dataframes(wonky_dfs_joined)
wonky_map = wonky_spark.toPandas()

print(f"Wonky map shape: {wonky_map.shape}")
print(f"Unique respondents in wonky studies: {wonky_map['respondent_pk'].nunique():,}")
print(f"Unique tasks in wonky studies: {wonky_map['task_pk'].nunique():,}")


In [0]:
tasks = task.toPandas()

In [0]:
user_info_df['taskPk']

In [0]:
user_info_df = user_info_df.merge(tasks[['task_pk', 'task_length_of_task']], how='left', left_on=['taskPk'], right_on=['task_pk'])

In [0]:
# get respondent level df from mapped balance tables at task level
wonky_respondent_df = (
    wonky_map.groupby(wonky_config['cols_to_group'])
    .agg({'uuid': 'count'}) 
    .reset_index()
    .rename(columns={"uuid": "wonky_study_count", "respondent_pk": "balance_respondentPk"})
)

wonky_respondent_df = wonky_respondent_df[
    [col for col in wonky_respondent_df.columns if col not in wonky_config['cols_to_drop']]
]

wonky_respondent_df.display()

In [0]:
# summaries respondent level df (aggregate level) ['wonky_study_count_sum'] = 'total_wonky_studies', rename_dict['task_pk_count'] = 'wonky_task_instances' 
wonky_respondent_summary = create_wonky_respondent_summary(
    wonky_respondent_df,
    respondent_id_col="balance_respondentPk"
)

wonky_respondent_summary.display()

In [0]:
wonky_respondent_summary

In [0]:
task_completed = (
    user_info_df[['respondentPk', 'taskPk']]
    .groupby('respondentPk')
    .count()
    .rename(columns={'taskPk': 'task_completed'})
    .reset_index()
)

wonky_counts = calculate_wonky_task_ratio(
    task_completed,
    wonky_respondent_summary
)

wonky_counts['wonky_task_ratio'] = np.where(wonky_counts['wonky_task_ratio'] > 1, 1, wonky_counts['wonky_task_ratio'])

print(f"Aggregated (user level) final shape: {wonky_counts.shape}")
print(f"Respondents in wonky_counts: {len(wonky_counts):,}")
print(f"Avg total tasks per respondent: {wonky_counts['task_completed'].mean():.2f}")
print(f"Avg wonky tasks per respondent: {wonky_counts['wonky_task_instances'].mean():.2f}")
print(f"Avg wonky task ratio: {wonky_counts['wonky_task_ratio'].mean():.2%}")


In [0]:
wonky_counts['wonky_task_ratio'].max()

In [0]:
wonky_counts['wonky_task_ratio'].min()

In [0]:
wonky_counts['wonky_task_ratio'].median()

In [0]:
list_of_wonkyUsers = list(wonky_map['respondent_pk'].unique())
task_completed['wonky_flag'] = np.where(task_completed['respondentPk'].isin(list_of_wonkyUsers), 1, 0)

task_completed.sort_values('task_completed', ascending=False).head(30)

In [0]:
wonky_map[wonky_map['respondent_pk']  == '65e2c4dc-097c-4088-a43d-0578ebab6359']

In [0]:
user_info_df_vstudy[(user_info_df_vstudy['taskPk'] == '0e91fff6-05eb-4320-a53b-c260ff053547') & (user_info_df['respondentPk']  == '65e2c4dc-097c-4088-a43d-0578ebab6359')][['date_completed', 'wonky_study_flag']]

In [0]:
special_user = user_info_df[user_info_df['respondentPk']  == '65e2c4dc-097c-4088-a43d-0578ebab6359']
special_user['date_completed'] = pd.to_datetime(special_user['date_completed'], format='%Y-%m-%d').dt.date

special_user_chartdata = special_user[['date_completed', 'taskPk']].groupby('date_completed').count().reset_index()

In [0]:
fig = px.line(
    special_user_chartdata,
    x="date_completed",
    y="taskPk",
    markers=True  # optional: show markers on each date
)

fig.update_layout(
    xaxis_title="Date completed",
    yaxis_title="Number of tasks",
)

fig.show()

In [0]:
fig = px.histogram(task_completed, x="task_completed", nbins=30, histnorm='percent')
fig.update_layout(
    xaxis_title="Tasks completed by users since 11-09-25",
    yaxis_title="% of users completing tasks",
    bargap=0.05
)
fig.show()


In [0]:
print(f"\nTask time validation:")
print(f"  - Null task_time_taken_s: {user_info_df['task_time_taken_s'].isnull().sum()}")
print(f"  - Avg task time: {user_info_df['task_time_taken_s'].mean():.2f} seconds")

In [0]:
print(f"Wonky map shape: {wonky_map.shape}")
print(f"Unique respondents in wonky studies: {wonky_map['respondent_pk'].nunique():,}")
print(f"Unique tasks in wonky studies: {wonky_map['task_pk'].nunique():,}")

In [0]:
print(f"Wonky study records in Spark: {wonky_spark.count():,}")
display(wonky_map.head())

In [0]:
# Merge wonky data with user info (to include summarised wonky info at user level)
user_info_df_vstudy = merge_wonky_data_with_user_info(
    user_info_df,
    wonky_respondent_summary,
    left_on=["respondentPk", "taskPk"],
    right_on=["balance_respondentPk", "task_pk"]
)

# Add wonky flag to signal users that ever been involved in a wonky study
wonky_respondent_list = wonky_counts['respondentPk'].unique().tolist()
user_info_df_vstudy["wonky_study_flag"] = np.where(
    user_info_df_vstudy["respondentPk"].isin(wonky_respondent_list), 1, 0
)

print(f"Final user_info_df_vstudy shape: {user_info_df_vstudy.shape}")
print(f"Records with wonky flag: {(user_info_df_vstudy['wonky_study_flag']==1).sum():,}")

In [0]:
user_info_df_vstudy = user_info_df_vstudy.sort_index(axis=1)

In [0]:
merge_stats = user_info_df_vstudy['_merge'].value_counts()
print(f"Merge results:")
print(f"left_only (no wonky match): {merge_stats.get('left_only', 0):,}")
print(f"both (wonky match at task level): {merge_stats.get('both', 0):,}")

print(f"Hierarchical structure validation:")
print(f"Total task-level records: {len(user_info_df_vstudy):,}")
print(f"Unique respondents: {user_info_df_vstudy['respondentPk'].nunique():,}")
print(f"Unique tasks: {user_info_df_vstudy['taskPk'].nunique():,}")
print(f"Tasks with wonky match: {user_info_df_vstudy['wonky_task_instances'].notna().sum():,}")
print(f"Wonky study flag=1: {(user_info_df_vstudy['wonky_study_flag']==1).sum():,}")

user_info_df_vstudy = user_info_df_vstudy.drop(columns=['_merge'])

print(f"Final user_info_df_vstudy shape: {user_info_df_vstudy.shape}")
print("="*80)

In [0]:

user_info_df_vstudy[user_info_df_vstudy['wonky_study_flag'] == 1].to_csv('export.csv')

#### Print checks

In [0]:
print("="*80)
print("DATA QUALITY VALIDATION CHECKS")
print("="*80)

print("1. DUPLICATE CHECKS:")
task_level_dups = user_info_df_vstudy.duplicated(subset=['respondentPk', 'taskPk']).sum()
print(f"Task-level duplicates (respondentPk + taskPk): {task_level_dups}")
if task_level_dups > 0:
    print(f"WARNING: Found duplicates! Investigate before modeling.")

print("2. NULL VALUE CHECKS (Key columns):")
key_cols = ['respondentPk', 'taskPk', 'wonky_study_flag', 'task_completed', 'task_time_taken_s']
for col in key_cols:
    if col in user_info_df_vstudy.columns:
        null_count = user_info_df_vstudy[col].isnull().sum()
        print(f"   - {col}: {null_count:,} nulls ({null_count/len(user_info_df_vstudy)*100:.2f}%)")

print("\3. WONKY FLAG CONSISTENCY:")
flag_1_with_details = ((user_info_df_vstudy['wonky_study_flag']==1) & 
                        (user_info_df_vstudy['wonky_task_instances'].notna())).sum()
flag_1_without_details = ((user_info_df_vstudy['wonky_study_flag']==1) & 
                           (user_info_df_vstudy['wonky_task_instances'].isna())).sum()
print(f"Flag=1 with wonky details: {flag_1_with_details:,}")
print(f"Flag=1 WITHOUT wonky details: {flag_1_without_details:,}")
if flag_1_without_details > 0:
    print(f"Note: These respondents had wonky tasks but not in the current task set")

print("4. TASK TIME VALIDATION:")
negative_times = (user_info_df_vstudy['task_time_taken_s'] < 0).sum()
zero_times = (user_info_df_vstudy['task_time_taken_s'] == 0).sum()
print(f"Negative task times: {negative_times:,}")
print(f"Zero task times: {zero_times:,}")
print(f"Min time: {user_info_df_vstudy['task_time_taken_s'].min():.2f}s")
print(f"Max time: {user_info_df_vstudy['task_time_taken_s'].max():.2f}s")
print(f"Median time: {user_info_df_vstudy['task_time_taken_s'].median():.2f}s")

print("5. RESPONDENT-LEVEL VALIDATION:")
respondent_task_counts = user_info_df_vstudy.groupby('respondentPk')['taskPk'].count()
print(f"Min tasks per respondent: {respondent_task_counts.min()}")
print(f"Max tasks per respondent: {respondent_task_counts.max()}")
print(f"Median tasks per respondent: {respondent_task_counts.median()}")
print(f"Mean tasks per respondent: {respondent_task_counts.mean():.2f}")

print("6. WONKY COUNTS VALIDATION:")
print(f"Respondents in wonky_counts: {len(wonky_counts):,}")
print(f"Min wonky_task_ratio: {wonky_counts['wonky_task_ratio'].min():.2%}")
print(f"Max wonky_task_ratio: {wonky_counts['wonky_task_ratio'].max():.2%}") # might need to cap at 1
print(f"Mean wonky_task_ratio: {wonky_counts['wonky_task_ratio'].mean():.2%}")

print("\n" + "="*80)
print("DATA QUALITY CHECKS COMPLETE")
print("="*80)

In [0]:
print("="*80)
print("FINAL DATA EXPORT SUMMARY")
print("="*80)

print(f"1. user_info_df_vstudy (TASK-LEVEL - Master dataset for fraud modeling)")
print(f"Shape: {user_info_df_vstudy.shape}")
print(f"Task-level records: {len(user_info_df_vstudy):,}")
print(f"Unique respondents: {user_info_df_vstudy['respondentPk'].nunique():,}")
print(f"Unique tasks: {user_info_df_vstudy['taskPk'].nunique():,}")
print(f"Records with wonky flag: {(user_info_df_vstudy['wonky_study_flag']==1).sum():,}")
print(f"Records with wonky details: {user_info_df_vstudy['wonky_task_instances'].notna().sum():,}")

print(f"2. wonky_counts (RESPONDENT-LEVEL - Aggregated wonky metrics)")
print(f"Shape: {wonky_counts.shape}")
print(f"Unique respondents: {len(wonky_counts):,}")
print(f"Columns: {list(wonky_counts.columns)}")

print(f"\n{'='*80}")
print("HIERARCHICAL STRUCTURE FOR FRAUD MODELING:")
print(f"{'='*80}")
print("user_info_df_vstudy maintains TASK-LEVEL granularity")
print("Each row = one task completion by one respondent")
print("Includes wonky_study_flag (binary: 0/1)")
print("Includes wonky_studies_count and metadata when task is wonky")
print("Use this for modeling fraud at task-level")
print("")
print("wonky_counts provides RESPONDENT-LEVEL aggregation")
print("Each row = one respondent with wonky activities")
print("Includes task_completed (total tasks)")
print("Includes wonky_task_instances, wonky_unique_tasks, total_wonky_studies")
print("Includes wonky_task_ratio")
print("Use this for respondent-level fraud patterns")
print(f"{'='*80}")

### Saving

In [0]:
user_info_df_vstudy

In [0]:
wonky_counts

In [0]:
(wonky_respondent_df[['balance_respondentPk', 'task_targeting_type', 'exposure_band']].groupby(['task_targeting_type', 'exposure_band']).count()/len(wonky_respondent_df)).reset_index().rename({'balance_respondentPk': 'share_of_users(%)'}, axis=1).sort_values('share_of_users(%)', ascending=False)

In [0]:
wonky_respondent_df

In [0]:
sorted(user_info_df_vstudy.columns)

In [0]:
wonky_respondent_summary

In [0]:
wonky_respondent_summary = wonky_respondent_summary.merge(
    user_info_df_vstudy[["respondentPk", "task_completed"]].drop_duplicates(),
    left_on=["balance_respondentPk"],
    right_on=["respondentPk"],
    how="left",
)

In [0]:
wonky_respondent_summary

In [0]:
summary1 = wonky_respondent_summary[['exposure_band', 'wonky_task_instances', 'task_completed']].groupby(['exposure_band']).sum().reset_index()
summary1['wonky_task_ratio'] = summary1['wonky_task_instances'] / summary1['task_completed']

In [0]:
summary1['wonky_task_share(%)'] = summary1['wonky_task_instances'] / len(wonky_respondent_df)
summary1['task_completed(%)'] = summary1['task_completed'] / summary1['task_completed'].sum()

In [0]:
summary1

In [0]:
import os

notebook_path = os.getcwd() 
repo_root = os.path.abspath(os.path.join(notebook_path, ".."))
misc_dir = os.path.join(repo_root, "misc")
os.makedirs(misc_dir, exist_ok=True)

output_path = os.path.join(misc_dir,
                           os.path.basename(paths_config['output_files']['user_info_df']))
wonky_counts_path = os.path.join(misc_dir,
                          os.path.basename(paths_config['output_files']['wonky_user_counts']))
wonky_respondent_df_path = os.path.join(misc_dir,
                          os.path.basename(paths_config['output_files']['wonky_respondent_df']))
wonky_respondent_summary_path = os.path.join(misc_dir,
                          os.path.basename(paths_config['output_files']['wonky_respondent_summary']))
                          

wonky_counts.to_parquet(wonky_counts_path, index=False)
wonky_respondent_df.to_parquet(wonky_respondent_df_path, index=False)
wonky_respondent_summary.to_parquet(wonky_respondent_summary_path, index=False)

user_info_df_vstudy = user_info_df_vstudy.loc[:, ~user_info_df_vstudy.columns.duplicated()]
user_info_df_vstudy.to_parquet(output_path, index=False)

print("Files saved successfully:")
print(f"  - {output_path}")
print(f"  - {wonky_counts_path}")
print(f"  - {wonky_respondent_df_path}")
print(f"  - {wonky_respondent_summary_path}")

In [0]:
user_info_df_vstudy['date_completed'].min(), user_info_df_vstudy['date_completed'].max(), 

In [0]:
user_info_df_vstudy